In [1]:
import numpy as np 
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import pickle


In [2]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

with open(f'{data_dir}df_od_full.pkl', 'rb') as file:
    df_full = pickle.load(file)

fix_dict = {'Habur border crossing' : 'Habur Port', 'Basra Basra' : 'Basra', 'Basrah' : 'Basra', 'Baghdad05' : 'Baghdad','Arab jabour' : 'Baghdad',
            'Arab Jabour': 'Baghdad', 'Hay al Waahda' : 'Bagdad', 'Taqqadum' : 'Al Taqaddum', 'Ouwja' : 'Al-Awja', 'Husayba' : 'Husaybah', 
            "no-man's land": 'Al Karamah Border Crossing', 'Zacho' : 'Zaxo'}

df_full.replace(fix_dict, inplace= True)
cities = df_full['city'].unique()[1:]#You do not take nan

In [3]:
latitude = []
longitude = []
geo_countries = []

latitude_irq = []
longitude_irq = []
geo_countries_irq = []

for i in cities: 
    geolocator = Nominatim(user_agent='spm@ifs.ku.dk')
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    try: # the idea here is to catch both timeout and non-returns
        location = geocode(i)

        latitude.append(location.latitude)
        longitude.append(location.longitude)
        geo_countries.append(location.address.split(',')[-1])
    
    except:

        latitude.append(None)
        longitude.append(None)
        geo_countries.append(None)

    try:

        location_irq = geocode(f'{i}, Iraq')

        latitude_irq.append(location_irq.latitude)
        longitude_irq.append(location_irq.longitude)
        geo_countries_irq.append(location_irq.address.split(',')[-1])

    except:

        latitude_irq.append(None)
        longitude_irq.append(None)
        geo_countries_irq.append(None)

df_cities = pd.DataFrame({'city' : cities, 'latitude' : latitude, 'longitude' : longitude, 'geo_country': geo_countries, 
                          'latitude_irq' : latitude_irq, 'longitude_irq' : longitude_irq, 'geo_country_irq': geo_countries_irq })

In [17]:
translation_dict = {' العراق' : 'Iraq', ' الأردن' : 'Jordan', ' السعودية' : 'Saudi Arabia', ' سوريا' : 'Syria', ' Türkiye' : 'Turkey' }
df_cities.replace(translation_dict, inplace = True)

In [18]:
df_cities

,city,latitude,longitude,geo_country,latitude_irq,longitude_irq,geo_country_irq
0,Baghdad,33.302431,44.378799,Iraq,33.302431,44.378799,Iraq
1,Basra,30.495237,47.809098,Iraq,30.495237,47.809098,Iraq
2,Husaybah,33.420684,43.453298,Iraq,33.420684,43.453298,Iraq
3,Baquba,33.743727,44.649523,Iraq,33.743727,44.649523,Iraq
4,Karabilah,34.390889,41.046437,Iraq,34.390889,41.046437,Iraq
5,Mosul,36.343694,43.099716,Iraq,36.343694,43.099716,Iraq
6,Ad Dawr,34.458349,43.795799,Iraq,34.458349,43.795799,Iraq
7,Hilla,32.482245,44.432813,Iraq,32.482245,44.432813,Iraq
8,Samarra,34.200000,43.900000,Iraq,34.200000,43.900000,Iraq
9,Dujail,33.837069,44.247461,Iraq,33.837069,44.247461,Iraq


In [19]:
for i in range(0, df_cities.shape[0]):

    if df_cities.iloc[i]['latitude'] != df_cities.iloc[i]['latitude_irq'] or df_cities.iloc[i]['longitude'] != df_cities.iloc[i]['longitude_irq']:
        print(f'{df_cities.iloc[i]["city"]} {df_cities.iloc[i]["geo_country"]}')
        print()
        print('\n')


Al-Awja  Palestinian Territory



Al Karamah Border Crossing Jordan



Al-Samawa Saudi Arabia



Kara-Ula  Palestinian Territory



Al Asad Syria



Cizre Turkey



Sirnak Turkey



cizre Turkey



Habur Port Turkey



Manhattan  United States



